In [109]:
from nba_api.stats.endpoints import playerindex
import pandas as pd
import sqlconfig
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
#Finding all NBA players
def player_team():
    try:
        players=playerindex.PlayerIndex(season="2024-25")
        nba_players=players.get_dict()
        print("Fetched Player Data")
    except requests.exceptions.Timeout:
        print("Time out Error")
        nba_players=[]
        
    #Data Cleaning Dictonary to dataframe
    nba_players=pd.DataFrame(nba_players['resultSets'][0]['rowSet'], columns=nba_players['resultSets'][0]['headers'])
    
    #Extracting columns for BCNF

    #Active_NBA_Players
    Active_NBA_Players=nba_players[['PERSON_ID','TEAM_ID','PLAYER_LAST_NAME',"PLAYER_FIRST_NAME","JERSEY_NUMBER","POSITION","HEIGHT","WEIGHT","COLLEGE","COUNTRY","DRAFT_YEAR","DRAFT_ROUND","DRAFT_NUMBER","PTS","REB","AST"]]
    Active_NBA_Players = Active_NBA_Players.copy()
    Active_NBA_Players['PLAYER_PICTURE_URL'] = Active_NBA_Players['PERSON_ID'].apply(
        lambda x: f"https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/{x}.png"
    )
    Active_NBA_Players.columns=['player_id','team_id','player_last_name','player_first_name','jersey_number','position','player_height','player_weight',
                            'college','country','draft_year','draft_round','draft_number','pts','reb','ast','player_picture_url']
    
    Active_NBA_Players['team_id']=Active_NBA_Players['team_id'].astype(str)
    print(Active_NBA_Players.head(5))
    return Active_NBA_Players

In [ ]:
#SELECTING all active NBA teams so no players that are not in a NBA team does not get included in the database
def selectTeams():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = "SELECT n.team_id FROM nba24_25.nbateams n;"

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Identifying all players on active teams
def players_on_active_teams(Active_NBA_Players,teams):
    Active_NBA_Players=Active_NBA_Players[Active_NBA_Players['team_id'].isin(teams['team_id'])]
    return Active_NBA_Players

In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
Active_NBA_Players=player_team()
teams=selectTeams()

Fetched Player Data
   player_id     team_id player_last_name player_first_name jersey_number  \
0    1630173  1610612752          Achiuwa          Precious             5   
1     203500  1610612745            Adams            Steven            12   
2    1628389  1610612748          Adebayo               Bam            13   
3    1630534  1610612761           Agbaji             Ochai            30   
4    1630583  1610612763           Aldama             Santi             7   

  position player_height player_weight          college      country  \
0        F           6-8           243          Memphis      Nigeria   
1        C          6-11           265       Pittsburgh  New Zealand   
2      C-F           6-9           255         Kentucky          USA   
3        G           6-5           215           Kansas          USA   
4      F-C           7-0           215  Loyola-Maryland        Spain   

   draft_year  draft_round  draft_number   pts   reb  ast  \
0      2020.0          

,player_id,team_id,player_last_name,player_first_name,jersey_number,position,player_height,player_weight,college,country,draft_year,draft_round,draft_number,pts,reb,ast,player_picture_url
0,1630173,1610612752,Achiuwa,Precious,5,F,6-8,243,Memphis,Nigeria,2020.0,1.0,20.0,7.0,6.1,1.1,https://ak-static.cms.nba.com/wp-content/uploa...
1,203500,1610612745,Adams,Steven,12,C,6-11,265,Pittsburgh,New Zealand,2013.0,1.0,12.0,3.5,5.1,1.0,https://ak-static.cms.nba.com/wp-content/uploa...
2,1628389,1610612748,Adebayo,Bam,13,C-F,6-9,255,Kentucky,USA,2017.0,1.0,14.0,16.9,10.0,4.3,https://ak-static.cms.nba.com/wp-content/uploa...
3,1630534,1610612761,Agbaji,Ochai,30,G,6-5,215,Kansas,USA,2022.0,1.0,14.0,10.1,3.7,1.6,https://ak-static.cms.nba.com/wp-content/uploa...
4,1630583,1610612763,Aldama,Santi,7,F-C,7-0,215,Loyola-Maryland,Spain,2021.0,1.0,30.0,12.8,6.7,2.7,https://ak-static.cms.nba.com/wp-content/uploa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,203469,1610612745,Zeller,Cody,40,F-C,6-11,240,Indiana,USA,2013.0,1.0,4.0,NaN,NaN,NaN,https://ak-static.cms.nba.com/wp-content/uploa...
557,1627826,1610612746,Zubac,Ivica,40,C,7-0,240,Mega Basket,Croatia,2016.0,2.0,32.0,15.4,12.6,2.5,https://ak-static.cms.nba.com/wp-content/uploa...
558,1641783,1610612753,da Silva,Tristan,23,F,6-8,217,Colorado,Germany,2024.0,1.0,18.0,7.9,3.6,1.6,https://ak-static.cms.nba.com/wp-content/uploa...
559,1628427,1610612743,Čančar,Vlatko,31,F,6-8,236,San Pablo Burgos,Slovenia,2017.0,2.0,49.0,2.3,2.0,0.0,https://ak-static.cms.nba.com/wp-content/uploa...


In [ ]:
Active_NBA_Players=players_on_active_teams(Active_NBA_Players,teams)

,player_id,team_id,player_last_name,player_first_name,jersey_number,position,player_height,player_weight,college,country,draft_year,draft_round,draft_number,pts,reb,ast,player_picture_url
0,1630173,1610612752,Achiuwa,Precious,5,F,6-8,243,Memphis,Nigeria,2020.0,1.0,20.0,7.0,6.1,1.1,https://ak-static.cms.nba.com/wp-content/uploa...
1,203500,1610612745,Adams,Steven,12,C,6-11,265,Pittsburgh,New Zealand,2013.0,1.0,12.0,3.5,5.1,1.0,https://ak-static.cms.nba.com/wp-content/uploa...
2,1628389,1610612748,Adebayo,Bam,13,C-F,6-9,255,Kentucky,USA,2017.0,1.0,14.0,16.9,10.0,4.3,https://ak-static.cms.nba.com/wp-content/uploa...
3,1630534,1610612761,Agbaji,Ochai,30,G,6-5,215,Kansas,USA,2022.0,1.0,14.0,10.1,3.7,1.6,https://ak-static.cms.nba.com/wp-content/uploa...
4,1630583,1610612763,Aldama,Santi,7,F-C,7-0,215,Loyola-Maryland,Spain,2021.0,1.0,30.0,12.8,6.7,2.7,https://ak-static.cms.nba.com/wp-content/uploa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,203469,1610612745,Zeller,Cody,40,F-C,6-11,240,Indiana,USA,2013.0,1.0,4.0,NaN,NaN,NaN,https://ak-static.cms.nba.com/wp-content/uploa...
557,1627826,1610612746,Zubac,Ivica,40,C,7-0,240,Mega Basket,Croatia,2016.0,2.0,32.0,15.4,12.6,2.5,https://ak-static.cms.nba.com/wp-content/uploa...
558,1641783,1610612753,da Silva,Tristan,23,F,6-8,217,Colorado,Germany,2024.0,1.0,18.0,7.9,3.6,1.6,https://ak-static.cms.nba.com/wp-content/uploa...
559,1628427,1610612743,Čančar,Vlatko,31,F,6-8,236,San Pablo Burgos,Slovenia,2017.0,2.0,49.0,2.3,2.0,0.0,https://ak-static.cms.nba.com/wp-content/uploa...


In [ ]:
intitalInsertion(Active_NBA_Players,"active_nba_players")

Data inserted
